In [32]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt

In [33]:
CELL_LENGTH = 21.34
ROWS = 9
COLS = 6

In [34]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((COLS * ROWS, 3), np.float32)
objp[:,:2] = np.mgrid[0:ROWS,0:COLS].T.reshape(-1,2) * CELL_LENGTH
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('images/webcam/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (ROWS, COLS), None)
    print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        print(corners)
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (ROWS, COLS), corners2, ret)
        plt.figure(figsize = (20,15))
        #plt.imshow(img)
        #plt.show()
        # cv.imshow('img', img)
        # cv.waitKey(0)



cv.destroyAllWindows()

True
[[[ 395.       440.     ]]

 [[ 493.5967   416.08563]]

 [[ 583.       395.5    ]]

 [[ 668.06793  374.95288]]

 [[ 748.6499   357.73773]]

 [[ 822.70294  339.37427]]

 [[ 892.       323.5    ]]

 [[ 960.12036  306.7132 ]]

 [[1022.       293.5    ]]

 [[ 417.89127  530.5802 ]]

 [[ 511.52228  506.09723]]

 [[ 599.83356  481.72284]]

 [[ 679.9158   460.22504]]

 [[ 760.7381   439.87247]]

 [[ 833.5044   421.56247]]

 [[ 902.7922   402.21588]]

 [[ 966.9802   385.92743]]

 [[1028.9478   368.04645]]

 [[ 439.52335  619.9456 ]]

 [[ 530.4832   590.82336]]

 [[ 614.1252   566.3446 ]]

 [[ 696.3683   541.3206 ]]

 [[ 772.3669   520.7466 ]]

 [[ 842.9561   499.9561 ]]

 [[ 909.4967   478.27908]]

 [[ 975.073    458.5373 ]]

 [[1034.06     441.48868]]

 [[ 459.       703.5    ]]

 [[ 547.1405   673.47986]]

 [[ 631.50946  644.8829 ]]

 [[ 709.       620.     ]]

 [[ 783.6916   594.89594]]

 [[ 854.33984  574.3895 ]]

 [[ 919.9004   550.56555]]

 [[ 980.4891   530.79553]]

 [[1040.3036   

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

In [35]:
corners

In [36]:
#returns cam matrix, d, rotv trv
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img = cv.imread('images/webcam/00.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
#plt.imshow(dst)
cv.imwrite('calibresult.png', dst)

True

In [37]:
#Get difference between actual points and projected points
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i],rvecs[i],tvecs[i],mtx,dist)
    error = cv.norm(imgpoints[i],imgpoints2,cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print ("Total error: {}".format(mean_error/len(objpoints)))
#Closer to 0 = better

Total error: 0.0999662978411672


In [38]:
# get coordinates of an image by clicking on it. Store the coordinates in a list
coordinates = []
def click_event(event, x, y, flags, params):
    if event == cv.EVENT_LBUTTONDOWN:
        print(x, ' ', y)
        coordinates.append((x, y))
        font = cv.FONT_HERSHEY_SIMPLEX
        cv.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv.imshow('image', img)
    if event == cv.EVENT_RBUTTONDOWN:
        cv.destroyAllWindows()

img = cv.imread('images/webcam/00.jpg')
cv.imshow('image', img)
cv.setMouseCallback('image', click_event)
cv.waitKey(0)
cv.destroyAllWindows()
